In [ ]:
import tensorflow as tf
import pandas as pd

# univariate multi-step lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from numpy import hstack
import numpy as np

#Multivariente 3K Datos

In [ ]:
Datos3K=read_csv('/content/drive/MyDrive/Colab Notebooks/3KoutfitNuevaForma80X80.csv',delimiter=',', index_col=False)
Datos3K = array(Datos3K)



In [ ]:
Datos3K.shape

(19200, 2994)

In [ ]:
Yvalues = []
for j in range(4):
  for i in range(2994): 
    Yvalues.append(i)
print(len(Yvalues))


11976


In [ ]:
parte1=(Datos3K[0:6400,:])
parte2=(Datos3K[6400:12800,:])
parte3=(Datos3K[12800:,:])
print(parte1.shape,parte2.shape,parte3.shape)

(6400, 2994) (6400, 2994) (6400, 2994)


In [ ]:
Data1=list()
Data1 = np.concatenate([parte1, parte2,parte3])
Data2 = np.concatenate([parte1, parte3,parte2])
Data3 = np.concatenate([parte3, parte2,parte1])
Data4 = np.concatenate([parte2, parte3,parte1])

Data1

array([[7.00029020e-01, 3.28870588e-03, 0.00000000e+00, ...,
        0.00000000e+00, 1.42593804e-01, 6.53861961e-01],
       [6.20381961e-01, 1.03142157e-02, 0.00000000e+00, ...,
        0.00000000e+00, 6.38556196e-01, 8.74270588e-01],
       [6.08190980e-01, 1.21725863e-01, 0.00000000e+00, ...,
        3.92156863e-03, 6.86160588e-01, 8.34623529e-01],
       ...,
       [6.05924706e-01, 7.80549020e-03, 2.19227059e-01, ...,
        0.00000000e+00, 0.00000000e+00, 5.65490196e-04],
       [5.91617647e-01, 4.19941176e-03, 2.36089804e-01, ...,
        0.00000000e+00, 0.00000000e+00, 5.65490196e-04],
       [6.01462353e-01, 8.46039216e-04, 2.34874118e-01, ...,
        0.00000000e+00, 0.00000000e+00, 5.65490196e-04]])

In [ ]:
Data1=Data1.transpose()
Data2=Data2.transpose()
Data3=Data3.transpose()
Data4=Data4.transpose()
DatosCompletos=np.concatenate([Data1, Data2,Data3,Data4])

In [ ]:
DatosCompletos=DatosCompletos.transpose()

In [ ]:
print(DatosCompletos.shape)

(19200, 11976)


In [ ]:
Yvalues= array(Yvalues)
Yvalues = Yvalues.reshape(1,11976) 
DatosCompletos=np.concatenate([DatosCompletos,Yvalues])

In [ ]:
DatosCompletos.shape

(19201, 11976)

In [ ]:
DatosCompletos

array([[7.00029020e-01, 3.28870588e-03, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 8.33333333e-04],
       [6.20381961e-01, 1.03142157e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 4.47215686e-03],
       [6.08190980e-01, 1.21725863e-01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 8.11098039e-03],
       ...,
       [5.91617647e-01, 4.19941176e-03, 2.36089804e-01, ...,
        2.55806980e-02, 7.41341176e-02, 0.00000000e+00],
       [6.01462353e-01, 8.46039216e-04, 2.34874118e-01, ...,
        1.69137255e-02, 9.15372549e-03, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 2.00000000e+00, ...,
        2.99100000e+03, 2.99200000e+03, 2.99300000e+03]])

In [ ]:
# Se divie los samples igual que en las entradas univariantes, pero las secuencias se arman con otras condiciones
def split_sequence(sequences,n_steps):
    X,y = list(),list()
    #print(len(sequences))
    for i in range(11976):

        #print("i",i) 
        # ordenar las partes de entrada y salida del patrón
        seq_x,seq_y = sequences[:-1,i],sequences[-1,i]
        # Las "x" están formadas por los pares de la seq1 y seq2.
        X.append(seq_x)
        #Las "y" son los valores de la columna final (out_seq)
        y.append(seq_y)
        #print("valores de las secuencias")
        #print("seq_x",seq_x)
        #print("seq_y",seq_y)

    return array(X),array(y)


# Número de pasos de tiempo
n_steps = 1

# convertir a entrada y salida (x e y)
X, y = split_sequence(DatosCompletos, n_steps)

# el dataset tiene 2 series paralelas y por tanto, se define 2 en los n_features
n_features = X.shape[1]

#print("info de x e y")
#print(X.shape, y.shape)
# summarize the data
#print("datos de x e y en arrays")
#for i in range(len(X)):
#	print(X[i], y[i])
 

In [ ]:
print(X.shape)
print(y.shape)

(11976, 19200)
(11976,)


In [ ]:
X =np.reshape(X,(X.shape[0],60,320))
X.shape

(11976, 60, 320)

In [ ]:
from keras.optimizers.optimizer_experimental.optimizer import learning_rate_schedule
from keras.callbacks import EarlyStopping
# definición del modelo
model = Sequential()
model.add(LSTM(50, return_sequences=True,
               input_shape=(60,320) , kernel_initializer='TruncatedNormal',activation='ReLU'))  # returns a sequence of vectors of dimension 32
model.add(LSTM(100, return_sequences=True,activation='ReLU'))  # returns a sequence of vectors of dimension 32
model.add(LSTM(150,return_sequences=True,activation='ReLU'))
model.add(LSTM(200,return_sequences=True,activation='ReLU'))
model.add(LSTM(100,return_sequences=True,activation='ReLU'))  # return a single vector of dimension 32

model.add(Dense(50, activation='ReLU'))
model.add(Dense(1, activation='Softmax'))
model.compile(optimizer='Adam',metrics=['accuracy'] , loss='Huber')
es = EarlyStopping(monitor='accuracy', mode='min', verbose=1)
# fit del modelo
model.fit(X, y, epochs=200,batch_size=200, verbose=1,validation_split=0.2)

Epoch 1/200
48/48 [==============================] - 107s 2s/step - loss: 1420.2202 - accuracy: 4.1754e-04 - val_loss: 1794.0000 - val_accuracy: 0.0000e+00
Epoch 2/200
48/48 [==============================] - 91s 2s/step - loss: 1420.2205 - accuracy: 4.1754e-04 - val_loss: 1794.0000 - val_accuracy: 0.0000e+00
Epoch 3/200
48/48 [==============================] - 93s 2s/step - loss: 1420.2196 - accuracy: 4.1754e-04 - val_loss: 1794.0000 - val_accuracy: 0.0000e+00
Epoch 4/200
48/48 [==============================] - 87s 2s/step - loss: 1420.2200 - accuracy: 4.1754e-04 - val_loss: 1794.0000 - val_accuracy: 0.0000e+00
Epoch 5/200
48/48 [==============================] - 101s 2s/step - loss: 1420.2202 - accuracy: 4.1754e-04 - val_loss: 1794.0000 - val_accuracy: 0.0000e+00
Epoch 6/200
48/48 [==============================] - 105s 2s/step - loss: 1420.2194 - accuracy: 4.1754e-04 - val_loss: 1794.0000 - val_accuracy: 0.0000e+00
Epoch 7/200
48/48 [==============================] - 91s 2s/step - 

In [ ]:
model.summary()